In [1]:
import time
start_time = time.time()

In [2]:
import pandas as pd
data1 = pd.read_csv("VerizonFinancialsCleaned.csv")
data1.head()

,Date,StockPrice,TTMNetEPS,TTMSalesperShare,PricetoSalesRatio,BookValueperShare,TTMFCFperShare,QuarterlyRevenue,GrossProfit,OperatingIncome,...,CurrentLiailities,CurrentAssetsInventory,QuickRatio,DebtToEquity Ratio,TTMNetIncome,LTInvestmentsDet,TotalAssets,CashOnHand,LongTermDebt,TotalLiabilities
0,6/30/2018,49.73,7.56,31.58,1.57,12.97,6.01,32203,18572,6617,...,32.33,29.77,0.92,2.04,26.40,162.75,263303,1750,109174,209728
1,3/31/2018,46.68,7.63,31.27,1.49,12.67,4.49,31772,18517,7349,...,31.62,29.74,0.94,2.15,27.80,165.08,264516,1923,112734,212171
2,12/31/2017,51.09,7.36,30.83,1.66,10.95,3.57,33955,19535,5230,...,33.04,28.88,0.87,2.54,27.41,158.33,257143,2079,113642,212456
3,9/30/2017,47.20,3.90,30.44,1.55,6.97,3.01,31717,19112,7208,...,28.93,29.59,1.02,4.05,30.21,143.77,254682,4487,115317,226231
4,6/30/2017,42.03,3.90,30.26,1.39,6.56,3.10,30548,18064,8014,...,27.76,27.71,1.00,4.35,29.54,143.15,252978,4583,116390,226217


In [3]:
import numpy as np
# normalize data with natural log of difference between each day, add as extra row
# lognormal = percentage change per day, but can add or subtract cumulatively, unlike percentage
data1['ln StockPrice']=np.log(data1.StockPrice/data1.StockPrice.shift(1))
data1.head()

,Date,StockPrice,TTMNetEPS,TTMSalesperShare,PricetoSalesRatio,BookValueperShare,TTMFCFperShare,QuarterlyRevenue,GrossProfit,OperatingIncome,...,CurrentAssetsInventory,QuickRatio,DebtToEquity Ratio,TTMNetIncome,LTInvestmentsDet,TotalAssets,CashOnHand,LongTermDebt,TotalLiabilities,ln StockPrice
0,6/30/2018,49.73,7.56,31.58,1.57,12.97,6.01,32203,18572,6617,...,29.77,0.92,2.04,26.40,162.75,263303,1750,109174,209728,NaN
1,3/31/2018,46.68,7.63,31.27,1.49,12.67,4.49,31772,18517,7349,...,29.74,0.94,2.15,27.80,165.08,264516,1923,112734,212171,-0.063293
2,12/31/2017,51.09,7.36,30.83,1.66,10.95,3.57,33955,19535,5230,...,28.88,0.87,2.54,27.41,158.33,257143,2079,113642,212456,0.090273
3,9/30/2017,47.20,3.90,30.44,1.55,6.97,3.01,31717,19112,7208,...,29.59,1.02,4.05,30.21,143.77,254682,4487,115317,226231,-0.079195
4,6/30/2017,42.03,3.90,30.26,1.39,6.56,3.10,30548,18064,8014,...,27.71,1.00,4.35,29.54,143.15,252978,4583,116390,226217,-0.116010


In [4]:
#create new dataframe copy the columns and normalize the StockPrice column
#lognormal = percentage change per day, but can add or subtract cumulatively, unlike percentage
data1norm = data1.filter(['Date'], axis=1)
data1norm['StockPrice'] = data1['StockPrice']
data1norm['Log Normal StockPrice']=np.log(data1.StockPrice/data1.StockPrice.shift(1))

In [5]:
data1norm.head()

,Date,StockPrice,Log Normal StockPrice
0,6/30/2018,49.73,NaN
1,3/31/2018,46.68,-0.063293
2,12/31/2017,51.09,0.090273
3,9/30/2017,47.20,-0.079195
4,6/30/2017,42.03,-0.116010


In [6]:
# check for all rows with null values eliminate at the end
nan = data1norm[data1norm.isnull().any(axis=1)]
nan

,Date,StockPrice,Log Normal StockPrice
0,6/30/2018,49.73,NaN


In [7]:
#drop the first row
#data1norm.drop(data1norm.index[0])

In [8]:
#create new dataframe for ratios
data1ratios = data1norm.filter(['Date'], axis=1)

#caclulate the ratios

#PE Ratio = StockPrice/TTMNetEPS
#Description: 
data1ratios['PE Ratio'] = (data1['StockPrice']/data1['TTMNetEPS']) #should we trim these?

#PricetoBook Ratio = StockPrice/BookValueperShare
#Description: 
data1ratios['PricetoBook Ratio'] = (data1['StockPrice'])/data1['BookValueperShare']

#Current Ratio = TotalAssets/TotalLiabilities
#Description: 
data1ratios['Current Ratio'] = (data1['TotalAssets']/data1['TotalLiabilities'])

#Shareholder Equity = TotalAssets - TotalLiabilities
#Description: 
data1ratios['Shareholder Equity'] = (data1['TotalAssets'] - data1['TotalLiabilities'])

#DebtToEquity Ratio = LongTermDebt - Shareholder Equity
#Description: 
data1ratios['DebtToEquity Ratio'] = (data1['LongTermDebt'] - data1ratios['Shareholder Equity'])

#ROI = TTMNetIncome/(LTInvestmentsDet + LongTermDebt)
#Description: 
data1ratios['ROI'] = (data1['TTMNetIncome']/(data1['LTInvestmentsDet'] + data1['LongTermDebt']))

data1ratios.head()

,Date,PE Ratio,PricetoBook Ratio,Current Ratio,Shareholder Equity,DebtToEquity Ratio,ROI
0,6/30/2018,6.578042,3.834233,1.255450,53575,55599,0.000241
1,3/31/2018,6.117955,3.684294,1.246711,52345,60389,0.000246
2,12/31/2017,6.941576,4.665753,1.210335,44687,68955,0.000241
3,9/30/2017,12.102564,6.771879,1.125761,28451,86866,0.000262
4,6/30/2017,10.776923,6.407012,1.118298,26761,89629,0.000253


In [9]:
# Livio make data frame for Net Income and Revenue as percentages here
#data1percentages = data1.filter(['Date'], axis=1)
#data1percentages['Net Income % Change'] = 
#data1percentages['Revenue % Change'] = 


In [10]:
# combine all preproccessed data to a final data frame
data1final = data1.filter(['Date'], axis=1)
data1final['StockPrice'] = data1['StockPrice']
data1final['Log Normal StockPrice'] = data1norm['Log Normal StockPrice']
data1final['TTMNetEPS'] = data1['TTMNetEPS']
data1final['TTMSalesperShare'] = data1['TTMSalesperShare']
data1final['PricetoSalesRatio'] = data1['PricetoSalesRatio']
data1final['BookValueperShare'] = data1['BookValueperShare']
data1final['TTMFCFperShare'] = data1['TTMFCFperShare']
data1final['QuarterlyRevenue'] = data1['QuarterlyRevenue']
data1final['GrossProfit'] = data1['GrossProfit']
data1final['CurrentAssetsInventory'] = data1['CurrentAssetsInventory']
data1final['OperatingIncome'] = data1['CurrentAssetsInventory']
data1final['QuickRatio'] = data1['QuickRatio']
data1final['DebtToEquity Ratio'] = data1['DebtToEquity Ratio']
data1final['TTMNetIncome'] = data1['TTMNetIncome']
data1final['LTInvestmentsDet'] = data1['LTInvestmentsDet']
data1final['TotalAssets'] = data1['TotalAssets']
data1final['CashOnHand'] = data1['CashOnHand']
data1final['LongTermDebt'] = data1['LongTermDebt']
data1final['TotalLiabilities'] = data1['TotalLiabilities']
data1final['PE Ratio'] = data1ratios['PE Ratio']
data1final['PricetoBook Ratio'] = data1ratios['PricetoBook Ratio']
data1final['Current Ratio'] = data1ratios['Current Ratio']
data1final['Shareholder Equity'] = data1ratios['Shareholder Equity']
data1final['DebtToEquity Ratio'] = data1ratios['DebtToEquity Ratio']
data1final['ROI'] = data1ratios['ROI']
#data1final = data1percentages['Net Income % Change']
#data1final = data1percentages['Revenue % Change']
data1final.head()

,Date,StockPrice,Log Normal StockPrice,TTMNetEPS,TTMSalesperShare,PricetoSalesRatio,BookValueperShare,TTMFCFperShare,QuarterlyRevenue,GrossProfit,...,LTInvestmentsDet,TotalAssets,CashOnHand,LongTermDebt,TotalLiabilities,PE Ratio,PricetoBook Ratio,Current Ratio,Shareholder Equity,ROI
0,6/30/2018,49.73,NaN,7.56,31.58,1.57,12.97,6.01,32203,18572,...,162.75,263303,1750,109174,209728,6.578042,3.834233,1.255450,53575,0.000241
1,3/31/2018,46.68,-0.063293,7.63,31.27,1.49,12.67,4.49,31772,18517,...,165.08,264516,1923,112734,212171,6.117955,3.684294,1.246711,52345,0.000246
2,12/31/2017,51.09,0.090273,7.36,30.83,1.66,10.95,3.57,33955,19535,...,158.33,257143,2079,113642,212456,6.941576,4.665753,1.210335,44687,0.000241
3,9/30/2017,47.20,-0.079195,3.90,30.44,1.55,6.97,3.01,31717,19112,...,143.77,254682,4487,115317,226231,12.102564,6.771879,1.125761,28451,0.000262
4,6/30/2017,42.03,-0.116010,3.90,30.26,1.39,6.56,3.10,30548,18064,...,143.15,252978,4583,116390,226217,10.776923,6.407012,1.118298,26761,0.000253


In [11]:
#drop the first row which contains null value
data1final = data1final.drop(data1final.index[0])

In [12]:
#highlight new columns
def highlight_cols(s):
    color = 'yellow'
    return 'background-color: %s' % color

data1final.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['Log Normal StockPrice', 'PE Ratio', 'PricetoBook Ratio',
                                                                  'Current Ratio', 'Shareholder Equity', 'ROI']])


,Date,StockPrice,Log Normal StockPrice,TTMNetEPS,TTMSalesperShare,PricetoSalesRatio,BookValueperShare,TTMFCFperShare,QuarterlyRevenue,GrossProfit,CurrentAssetsInventory,OperatingIncome,QuickRatio,DebtToEquity Ratio,TTMNetIncome,LTInvestmentsDet,TotalAssets,CashOnHand,LongTermDebt,TotalLiabilities,PE Ratio,PricetoBook Ratio,Current Ratio,Shareholder Equity,ROI
1,3/31/2018,46.68,-0.0632926,7.63,31.27,1.49,12.67,4.49,31772,18517,29.74,29.74,0.94,60389,27.8,165.08,264516,1923,112734,212171,6.11796,3.68429,1.24671,52345,0.000246238
2,12/31/2017,51.09,0.090273,7.36,30.83,1.66,10.95,3.57,33955,19535,28.88,28.88,0.87,68955,27.41,158.33,257143,2079,113642,212456,6.94158,4.66575,1.21034,44687,0.00024086
3,9/30/2017,47.2,-0.0791949,3.9,30.44,1.55,6.97,3.01,31717,19112,29.59,29.59,1.02,86866,30.21,143.77,254682,4487,115317,226231,12.1026,6.77188,1.12576,28451,0.000261647
4,6/30/2017,42.03,-0.11601,3.9,30.26,1.39,6.56,3.1,30548,18064,27.71,27.71,1,89629,29.54,143.15,252978,4583,116390,226217,10.7769,6.40701,1.1183,26761,0.00025349
5,3/31/2017,45.34,0.075806,3,30.26,1.5,6.14,3.92,29814,17767,25.96,25.96,0.97,87802,26.08,137.88,246731,4307,112839,221694,15.1133,7.38436,1.11293,25037,0.000230844
6,12/31/2016,49.12,0.0800766,3.22,30.84,1.59,5.9,5.34,32340,17978,25.19,25.19,0.83,81401,27.06,129.47,244180,2880,105433,220148,15.2547,8.32542,1.10916,24032,0.000256341
7,9/30/2016,47.28,-0.0381789,3.44,31.29,1.51,5.39,9.13,30937,18708,24.3,24.3,0.8,80756,28.78,124.72,239498,6441,102739,217515,13.7442,8.7718,1.10106,21983,0.000279788
8,6/30/2016,50.29,0.0617189,3.54,31.85,1.58,5.09,11.47,30532,18311,19.91,19.91,0.58,72178,29.78,113.67,231870,2857,92922,211126,14.2062,9.88016,1.09825,20744,0.000320092
9,3/31/2016,48.2,-0.0424472,4.41,32.26,1.49,4.93,12.73,32171,19559,22.47,22.47,0.68,83512,33.04,123.72,244587,5846,103615,224484,10.9297,9.77688,1.08955,20103,0.000318492
10,12/31/2015,40.69,-0.169377,4.37,32.15,1.27,4.38,13.3,34254,19547,21.1,21.1,0.6,85398,33.06,121.08,244175,4820,103240,226333,9.31121,9.28995,1.07883,17842,0.00031985


In [13]:
#export to csv
data1final.to_csv('Verizon PreProccessed.csv')
print("Preprocessing Completed In: %s seconds" % (time.time() - start_time))

Preprocessing Completed In: 3.6726834774017334 seconds
